In [1]:
# -U: 最新版のインストール
!pip install -U openai \
    langchain==0.3.12 \
    langchain-community \
    langchain-core \
    langchain-openai \
    langgraph \

# rag用ライブラリのインストール
!pip install -U chromadb \
    langchain-chroma \
    pypdf \
    pdfminer.six

  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_openai-0.3.32-py3-none-any.whl.metadata (2.4 kB)
  Using cached langgraph-0.6.7-py3-none-any.whl.metadata (6.8 kB)
  Using cached langsmith-0.2.11-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.75-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.73-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.71-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.70-py3-none-any

In [2]:
import os
from typing import Literal
from dotenv import load_dotenv

from langchain import hub
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader, PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI


# .envファイル読み込み
load_dotenv()

True

In [3]:
#config
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gpt-4o-mini-2024-07-18'
TEMPERATURE = 0.0

In [4]:
# client作成
chat = ChatOpenAI(openai_api_key = OPENAI_API_KEY,
                 openai_api_base = BASE_URL,
                 model_name=MODEL,
                 temperature=TEMPERATURE)

In [5]:
loader = DirectoryLoader(
    "../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PyPDFLoader,
    # loader_cls=PDFMinerLoader
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.11s/it]

Loaded 159 documents


In [6]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [7]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 埋め込み
embedding = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-small"
)

#ベクトルデータベースのChromaDBaに保存
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

In [8]:
# vectorstoreから必要な情報を読み出す
retriever = vectorstore.as_retriever()

# ユーザーが与えるプロンプトに加えて、
# rag_promptを追加してLLMに与えるように設定
rag_prompt = hub.pull("rlm/rag-prompt")

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | chat
    | StrOutputParser()
)

In [11]:
#promptを作成
prompt_txt = """臨床心理士が行うメンタルヘルスケアカウンセリングをシミュレーションし、その対話内容に相槌を含め、話し言葉のまま文字起こししてください。
会話は中途半端で終わらせず、きりが良い会話にしてください。
相槌は実際の対話を想定して細かく入れてください。
語感は固くならないようにしてください。

形式は以下のようにしてください。Aがカウンセラーで、Bがカウンセリングを受ける人です。
カウンセリングを受ける人の悩みは仕事以外にもあり、様々です。
以下の例は３回しか言葉を交わしていませんが、100文字程度の会話になるようにしてください。

[
 "Aが話す言葉",
 "Bが話す言葉",
 "Aが話す言葉",
 ...
]
"""

In [12]:
#レスポンスを得る
ret = rag_chain.invoke(prompt_txt)

RateLimitError: Error code: 429 - {'error': {'message': 'Token rate limit exceeded: 2457600 per 1 day'}}

In [ ]:
# レスポンスを表示
print(ret)